In [5]:
# 这些模块和包都是在逐步的探索中所需要的，然后全部汇总到这里，
#    并不是一开始就知道了 ^_^ ^_^ ^_^
# 1、导入模块和包
import pandas as pd    # 加载并处理csv文件
import datetime        # 利用datetime处理时间戳
import _pickle as cPickle    # 数据以二进制进行高效的储存到文件
from collections import defaultdict     # 利用Python设置稀疏矩阵的NULL位置的默认值
import scipy.sparse as ss     # 利用scipy构建稀疏矩阵
import scipy.io as sio    # 利用scipy储存评分矩阵
import numpy as np    # 利用numpy创建指定长度或形状的矩阵以及矩阵运算
from numpy.random import random    # numpy.random中的randn函数生成一些正态分布的随机数据
import time    # 利用Python内置模块，计算训练时迭代的时间
import json    # 将模型参数保存为json文件，加载模型参数json文件
import scipy    # 将储存加载的稀疏评分矩阵转换为numpy矩阵形式

In [6]:
# 3、数据处理——构建评分矩阵

# 设置数据储存位置
data_path = "./../dataset/amazon-ratings/"
train = pd.read_csv(data_path + "train.csv", sep = ",", nrows = 2000)

# 计算用户数量和电影数量
unique_user = train['UserId'].unique()
unique_item = train['ProductId'].unique()

user_num = unique_user.shape[0]
item_num = unique_item.shape[0]

print("the number of user:",user_num)
print("the number of product:",item_num)

the number of user: 1562
the number of product: 537


In [7]:
# 建立用户和物品的索引表
# 本数据集中user_id和item_id都已经是索引了,可以减1，将从1开始编码变成从0开始的编码
# 下面的代码更通用，可对任意编码的用户和物品重新索引
user_index = dict()    # 使用字典索引 保存用户索引
item_index = dict()    # 保存电影索引

for index, user in enumerate(unique_user):
    user_index[user] = index
    
# 重新编码活动索引字典  
# 序列函数，Python有一些有用的序列函数。
# enumerate函数，迭代一个序列时， 你可能想跟踪当前项的序号。 
# 因为这么做很常见， Python内建了一个 enumerate 函数， 可以返回 (i, value) 元组序列：
# 当你索引数据时， 使用 enumerate 的一个好方法是计算序列（唯一的） dict 映射到位置的值：

# 重新编码活动索引字典 
for index, item in enumerate(unique_item):
    item_index[item] = index
    
# 保存重新索引的用户索引表
cPickle.dump(user_index, open(data_path + "user_index.pkl", 'wb'))
# 保存重新索引的电影索引表
cPickle.dump(item_index, open(data_path + "item_index.pkl", 'wb'))


In [8]:
# 默认值，下面的逻辑很常见：
# 关于设定值， 常见的情况是在字典的值是属于其它集合， 如列表。
# collections 模块有一个很有用的类， defaultdict ，可以传递类型或函数以生成每个位置的默认值
# setdefault 方法就正是干这个的,这两个有着异曲同工之妙

# 倒排表
# 统计每个用户打过分的电影   / 每个电影被哪些用户打过分
user_items = defaultdict(set)
item_users = defaultdict(set)

# 用户-物品关系矩阵R, 稀疏矩阵，记录用户对每个电影的打分
# user_num —— 代表m行， item_num —— 代表n列，构建 m*n 矩阵
user_item_score = ss.dok_matrix((user_num, item_num))

# 扫描数据，进行记录
for row in train.index:    # csv_file.index > RangeIndex(start=0, stop=10000, step=1)
    # 获取数据的字段属性
    row_info = train.iloc[row]
    
    user_id = row_info["UserId"]
    item_id = row_info["ProductId"]
    rating = row_info["Rating"]
    
    # 当前用户索引，利用重新索引表
    current_user_index = user_index[user_id]
    # 当前电影索引，利用重新索引表
    current_item_index = item_index[item_id]
    
    # 倒排表
    user_items[current_user_index].add(current_item_index)    # 该用户对这个电影进行了打分
    item_users[current_item_index].add(current_user_index)    # 该电影被该用户打分
    
    # 当前用户对当前电影的评分
    user_item_score[current_user_index, current_item_index] = rating    


# 保存倒排表
# 每个用户打分的电影
cPickle.dump(user_items, open(data_path + "user_items.pkl", 'wb'))
# 对每个电影打过分的用户
cPickle.dump(item_users, open(data_path + "item_users.pkl", 'wb'))

# 保存评分矩阵，以便于后面使用
sio.mmwrite(data_path + "user_item_score", user_item_score)
